# Imports y config

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import torch
from monai.metrics import DiceMetric
import torch.nn.functional as F
from src.metrics.segmentation_bak import SegmentationMetrics

In [4]:
os.chdir('C:\\Users\\Usuario\\TFG\\digipanca\\')

# Test: Dice

In [16]:
# Crear valores aleatorios simulando segmentaciones (batch_size=4, clases=5, 512x512)
y_pred = torch.randint(0, 5, (4, 512, 512))  # Predicción como índices de clase
y_true = torch.randint(0, 5, (4, 512, 512))  # Ground truth como índices de clase

# Convertir a one-hot para ambas implementaciones
n_classes = 5
y_pred_one_hot = F.one_hot(y_pred, num_classes=n_classes).permute(0, 3, 1, 2).float()
y_true_one_hot = F.one_hot(y_true, num_classes=n_classes).permute(0, 3, 1, 2).float()

# ======================
# Cálculo con MONAI
# ======================
dice_metric = DiceMetric(include_background=True, reduction="none")  # "none" para obtener todas las clases
monai_dice = dice_metric(y_pred_one_hot, y_true_one_hot)

# ======================
# Cálculo con tu implementación
# ======================
_, my_dice = SegmentationMetrics.dice_coefficient(y_pred, y_true)  # Devuelve valores por clase en un diccionario

# ======================
# Comparación
# ======================
print("==> Comparación de Dice Score por clase:")
for i in range(n_classes):
    print(f"Clase {i}: MONAI = {monai_dice[:, i].mean().item():.4f}, Mi implementación = {my_dice[f'dice_class_{i}']:.4f}")

print(f"\n==> Dice medio:")
print(f"MONAI = {monai_dice.mean().item():.4f}, Mi implementación = {my_dice['dice_mean']:.4f}")

==> Comparación de Dice Score por clase:
Clase 0: MONAI = 0.2001, Mi implementación = 0.2001
Clase 1: MONAI = 0.2007, Mi implementación = 0.2007
Clase 2: MONAI = 0.1997, Mi implementación = 0.1997
Clase 3: MONAI = 0.2004, Mi implementación = 0.2004
Clase 4: MONAI = 0.1999, Mi implementación = 0.1999

==> Dice medio:
MONAI = 0.2002, Mi implementación = 0.2002


# Test: IoU

In [14]:
# Crear valores aleatorios simulando segmentaciones (batch_size=2, clases=3, 256x256)
y_pred = torch.randint(0, 3, (2, 256, 256))  # Predicción como índices de clase
y_true = torch.randint(0, 3, (2, 256, 256))  # Ground truth como índices de clase

# Convertir a one-hot
n_classes = 3
y_pred_one_hot = F.one_hot(y_pred, num_classes=n_classes).permute(0, 3, 1, 2).float()
y_true_one_hot = F.one_hot(y_true, num_classes=n_classes).permute(0, 3, 1, 2).float()

# ======================
# Cálculo de IoU con MONAI (manual)
# ======================
intersection = torch.sum(y_pred_one_hot * y_true_one_hot, dim=(2, 3))  # Intersección por clase
union = torch.sum(y_pred_one_hot, dim=(2, 3)) + torch.sum(y_true_one_hot, dim=(2, 3)) - intersection  # Unión
monai_iou = (intersection + 1e-6) / (union + 1e-6)  # Evitar división por 0

# ======================
# Cálculo con tu implementación
# ======================
_, my_iou = SegmentationMetrics.iou_score(y_pred, y_true)  # Devuelve valores por clase en un diccionario

# ======================
# Comparación
# ======================
print("==> Comparación de IoU por clase:")
for i in range(n_classes):
    monai_iou_value = monai_iou[:, i].mean().item()  # Promediar sobre el batch
    my_iou_value = my_iou[f"iou_class_{i}"]
    print(f"Clase {i}: MONAI = {monai_iou_value:.4f}, Mi implementación = {my_iou_value:.4f}")

print(f"\n==> IoU medio:")
print(f"MONAI = {monai_iou.mean().item():.4f}, Mi implementación = {my_iou['iou_mean']:.4f}")


==> Comparación de IoU por clase:
Clase 0: MONAI = 0.1989, Mi implementación = 0.1989
Clase 1: MONAI = 0.2006, Mi implementación = 0.2006
Clase 2: MONAI = 0.1981, Mi implementación = 0.1981

==> IoU medio:
MONAI = 0.1992, Mi implementación = 0.1992
